In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
from glob import glob
import pickle
from copy import deepcopy

In [3]:
ERROR_METRICS = [
    "Energy/Mass",
    "Maximum CFL",
    "Mean Sea Level",
    "Total Mass",
    "Mean Salin",
    "Frac Mass Err",
    "Salin Err",
    "Temp Err",
]

def plot_performance_vs_correctness(df, error_type, norm):
    
    df_plot = df.dropna(subset=["Cost", f"{error_type} ({norm})"])
    baseline_cost = df_plot[(df_plot['Configuration Number'] == 0)]['Cost'].iloc[0]
    df_plot = df_plot.assign(Improvement = baseline_cost/df_plot['Cost'])
    
    fig = px.scatter(
        df_plot,
        x="Improvement",
        y=f"{error_type} ({norm})",
        color = '32-bit %',
        hover_data = ['Configuration Number'],
        log_y = True, 
    )
    fig.update_traces(
        marker={
            'size' : 14,
            'opacity' : 0.5,
            'line_width' : 1,
            'line_color' : "black",
        }
    )
    if not df_plot[df_plot["Configuration Number"] == 1].empty:
        fig.add_trace(
            go.Scatter(
                x = df_plot[df_plot["Configuration Number"] == 1]["Improvement"],
                y = df_plot[df_plot["Configuration Number"] == 1][error_type],
                mode = "markers+text",
                marker_symbol = 'circle-open',
                marker_size = 14,
                marker_color = "black",
                marker_line_width = 3,
                marker_line_color = "black",
                name = "Uniform 32-bit",
                showlegend=False,
                text=["Uniform 32-bit"],
                textposition=["top right"],
                textfont_size=16
            )
        )

    fig.update_layout(
        title=dict(
            text = "MOM6",
            y = 0.98,
            x = 0.5,
            xanchor = 'center',
            yanchor = 'top',
            font_size=30,
        ),
        width = 600,
        height = 300,
        coloraxis_showscale = True,
        yaxis_tickformat = "f",
        xaxis_tickformat = ".1f",
        xaxis_ticksuffix = "x",
        xaxis_title = "",
        yaxis_title = "Relative Error",
        font_size = 20,
        font_family = "Times New Roman",
        legend = dict(
            bgcolor = "#E5ECF6",
            entrywidth = 130,
            orientation = "h",
            yanchor = "bottom",
            xanchor = "right",
            x = 1.0,
            y = 1.02,
            title_text = ""
        ),
        margin = dict(
            r = 0,
            t = 40,
            b = 0,
            l = 0,
        ),
    coloraxis={
        "cmin" : 0,
        "cmax" : 100,
        "colorbar" : {
            'thickness' : 18,
            'title' : "% 32-bit<br>(Hotspot)",
            'orientation' : 'h',
            'title_font_size' : 24,
            'y' : -0.6,
            'len' : 0.8
        },
    },
    )
    fig.add_vline(x=1.0, line_width=2, line_dash="dash", line_color="grey")
    fig.add_hline(y=0.25, line_width=2, line_dash="dash", line_color="grey")

    return fig, df_plot


def plot_label_frequency(df):

    fig = px.histogram(
        df,
        x = "Label",
    )
    fig.update_layout(
        font_size = 18
    )

    return fig


def get_MOM6_data(search_log_path):

    with open(search_log_path, "r") as f:
        search_log_lines = f.readlines()

    df_entire = []
    df_subset = []
    for line in search_log_lines:
        print(line)
        row = {}

        try:
            row['Configuration Number'] = int(line.split(":")[0])
        except ValueError:
            continue
        config_dir_path = os.path.join(os.path.dirname(search_log_path), f"{row['Configuration Number']:0>4}")

        config_path = glob(f"{config_dir_path}/config*")
        assert ( len(config_path) == 1)
        row['Configuration Path'] = config_path[0]

        float_count = 0
        double_count = 0
        with open(row['Configuration Path'], "r" ) as f:
            llines = f.readlines()
            for lline in llines:
                if ",4" in lline:
                    float_count += 1
                elif ",8" in lline:
                    double_count += 1

        row['32-bit %'] = 100*(float_count / (double_count + float_count))

        if "[PASSED]" in line:
            row['Label'] = "Passed"
        elif "error threshold was exceeded" in line:
            row['Label'] = "Exceeded Error Threshold"
        elif "(timeout)" in line:
            row['Label'] = "Timeout"
        elif "(runtime failure)" in line:
            row['Label'] = "Runtime Error"
        elif "(compilation error)" in line:
            row['Label'] = 'Compilation Error'
        elif "(plugin error)" in line:
            row['Label'] = 'Prose Plugin Error'
        else:
            continue

        try:
            with open(os.path.join(config_dir_path, "errors_L2.npy"), "rb") as f:
                errors = np.load(f)
            for i, metric in enumerate(ERROR_METRICS):
                row[f"{metric} (L2 norm)"] = errors[i]

            with open(os.path.join(config_dir_path, "errors_inf.npy"), "rb") as f:
                errors = np.load(f)
            for i, metric in enumerate(ERROR_METRICS):
                row[f"{metric} (inf norm)"] = errors[i]

        except FileNotFoundError:
            for metric in ERROR_METRICS:
                row[f"{metric} (L2 norm)"] = np.nan
                row[f"{metric} (inf norm)"] = np.nan
 
        try:
            row['Cost'] = float(line.split()[4])
        except ValueError:
            try:
                row['Cost'] = float(line.split()[5])
            except ValueError:
                row['Cost'] = np.nan

        df_entire.append(deepcopy(row))

        try:
            with open(os.path.join(config_dir_path, "gptl_subset_info.pckl"), "rb") as f:
                gptl_subset_info = pickle.load(f)

            import subprocess
            subprocess.run(
                f"tar -xvf gptl_timing.tar.gz",
                shell = True,
                cwd = config_dir_path,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )

            execution_counts = {}
            with open(os.path.join(config_dir_path, "timing.000000"), "r") as f:
                for line in f.readlines():
                    if line[2:].lstrip().startswith("::"):
                        procedure_name = line[2:].split()[0].strip().lower()
                        execution_count = float(line[2:].split()[1].strip())
                        execution_counts[procedure_name] = execution_count

            subprocess.run(
                f"rm timing.*",
                shell = True,
                cwd = config_dir_path,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )

            for key, value in gptl_subset_info.items():
                row['Procedure Name'] = key
                row['CPU Time'] = value
                try:
                    row["Execution Count"] = execution_counts[row['Procedure Name'].lower()]
                except KeyError:
                    row["Execution Count"] = np.nan
                float_count = 0
                double_count = 0
                config_hash = ""
                with open(row['Configuration Path'], "r" ) as f:
                    llines = f.readlines()
                    for lline in llines:
                        if lline.strip().lower().startswith(key.lower() + "::"):
                            if ",4" in lline:
                                float_count += 1
                                config_hash = config_hash + "4"
                            elif ",8" in lline:
                                double_count += 1
                                config_hash = config_hash + "8"
                
                row["config_hash"] = hash(config_hash)

                try:
                    row['32-bit %'] = 100*(float_count / (double_count + float_count))
                except Exception as e:
                    row['32-bit %'] = np.nan

                df_subset.append(deepcopy(row))

        except FileNotFoundError:
            continue

    return pd.DataFrame(df_entire), pd.DataFrame(df_subset)

In [4]:
df_entire, df_subset = get_MOM6_data("./prose_logs/__search_log_12hr.txt")

0000: [PASSED] subset cost = 6.310 (total cost = 407.219)

0001: [FAILED] (runtime failure) unable to execute transformed source code (see outlog.txt for details)

0002: [FAILED] (runtime failure) unable to execute transformed source code (see outlog.txt for details)

0003: [FAILED] subset cost = 37.440 (total cost = 765.834) but error threshold was exceeded

0004: [FAILED] (runtime failure) unable to execute transformed source code (see outlog.txt for details)

0005: [FAILED] subset cost = 31.882 (total cost = 712.011) but error threshold was exceeded

0006: [FAILED] (runtime failure) unable to execute transformed source code (see outlog.txt for details)

0007: [FAILED] subset cost = 17.613 (total cost = 505.293) but error threshold was exceeded

0008: [FAILED] (runtime failure) unable to execute transformed source code (see outlog.txt for details)

0009: [FAILED] subset cost = 37.497 (total cost = 847.684) but error threshold was exceeded

0010: [FAILED] subset cost = 31.374 (total c

In [21]:
fig, df_plot = plot_performance_vs_correctness(df_entire, error_type=ERROR_METRICS[1], norm="L2 norm")
fig.show()

In [29]:
df_subset[(df_subset["Procedure Name"] == procedure_name) & (df_subset["Configuration Number"] == 0)]["config_hash"].values[0]

-7709736075822909268

In [62]:
from plotly.subplots import make_subplots

df_subset["Average CPU Time Per Call"] = df_subset["CPU Time"] / df_subset["Execution Count"]
for procedure_name in df_subset[df_subset["Procedure Name"].str.contains("::")]["Procedure Name"].unique():
    if "flux_adjust" in procedure_name:
        continue # don't take the average of all 64-bit variants of flux-adjust routines which are noisy
    for config_hash in df_subset[df_subset["Procedure Name"] == procedure_name]["config_hash"].unique():
        df_subset.loc[(df_subset["Procedure Name"] == procedure_name) & (df_subset["config_hash"] == config_hash), "Average CPU Time Per Call"] = df_subset.loc[(df_subset["Procedure Name"] == procedure_name) & (df_subset["config_hash"] == config_hash), "Average CPU Time Per Call"].mean()

baseline_costs = {
    '::mom_continuity_ppm::zonal_flux_layer' : 1.4869062499999997,
    # '::mom_continuity_ppm::merid_flux_layer' : 1.3342421875000001,
    '::mom_continuity_ppm::zonal_mass_flux' : 0.5896171875,
    # '::mom_continuity_ppm::meridional_mass_flux' : 0.46635546875,
    '::mom_continuity_ppm::zonal_flux_adjust' : 0.5046796875,
    # '::mom_continuity_ppm::meridional_flux_adjust' : 0.39278515625,
    '::mom_continuity_ppm::set_zonal_bt_cont' : 0.41324609375,
    # '::mom_continuity_ppm::set_merid_bt_cont' : 0.35382031249999996,
    # '::mom_continuity_ppm::ppm_reconstruction_x' : 0.27198828125,
    # '::mom_continuity_ppm::ppm_reconstruction_y' : 0.22505468750000002,
    # '::mom_continuity_ppm::ppm_limit_pos' : 0.160015625,
    # '::mom_continuity_ppm::merid_face_thickness' : 0.15793749999999995,
    '::mom_continuity_ppm::zonal_face_thickness' : 0.15079687499999997,
    # '::mom_continuity_ppm::continuity_ppm' : 0.12741796875,
    # '::mom_continuity_ppm::continuity_ppm_init' : 1.10312109375e-05,
}

procedure_percentages = {
    '::mom_continuity_ppm::zonal_flux_layer': "22.4",#22.43212079470703,
    '::mom_continuity_ppm::merid_flux_layer': "20.1", #20.113111263774215,
    '::mom_continuity_ppm::zonal_mass_flux': "9.0",#8.9596406665791,
    '::mom_continuity_ppm::meridional_mass_flux': "7.0",#7.023666416682972,
    '::mom_continuity_ppm::zonal_flux_adjust': "7.6",#7.572937598608928,
    '::mom_continuity_ppm::meridional_flux_adjust': "5.9",#5.874188908917873,
    '::mom_continuity_ppm::set_zonal_bt_cont': "6.3",#6.251864356826961,
    '::mom_continuity_ppm::set_merid_bt_cont': "5.3",#5.334637994911223,
    '::mom_continuity_ppm::ppm_reconstruction_x': "4.1",#4.102172995990434,
    '::mom_continuity_ppm::ppm_reconstruction_y': "3.4",#3.398163531154783,
    '::mom_continuity_ppm::ppm_limit_pos': "2.4",#2.4479508889011354,
    '::mom_continuity_ppm::merid_face_thickness': "2.4",#2.401824905137172,
    '::mom_continuity_ppm::zonal_face_thickness': "2.3",#2.3120783396943727,
    '::mom_continuity_ppm::continuity_ppm': "2.0",#1.9538947557998065,
    '::mom_continuity_ppm::continuity_ppm_init': "<0.001",#0.00017002637389540776
}

fig = make_subplots(len(baseline_costs),1, subplot_titles=tuple([f'{x[x.rfind(":") + 1:]} ({procedure_percentages[x]}%)' for x in baseline_costs.keys()]), shared_xaxes=True, vertical_spacing=0.09)
for i, procedure_name in enumerate(baseline_costs.keys()):
    df_plot = df_subset[df_subset["Procedure Name"] == procedure_name]
    # baseline_cost = baseline_costs[procedure_name] / df_plot[df_plot["Configuration Number"] == 0]["Execution Count"].values[0]
    # baseline_cost = (baseline_cost + df_plot[df_plot["Configuration Number"] == 0]["Average CPU Time Per Call"].values[0]) / 2
    baseline_cost = df_plot[df_plot["Configuration Number"] == 0]["Average CPU Time Per Call"].values[0]
    df_plot = df_plot.assign(Improvement = np.round(baseline_cost/df_plot['Average CPU Time Per Call'], decimals=2))
    if "flux_adjust" in procedure_name: # reduce crowding of flux adjust variants with > 0.1x improvement
        df_plot.loc[df_plot["Improvement"] > 0.1, "Improvement"] = np.round(df_plot[df_plot["Improvement"] > 0.1]["Improvement"], decimals=1)
    df_plot = df_plot.drop_duplicates(subset=["config_hash","Improvement"])
    fig.add_trace(
        go.Scatter(
            x = df_plot["Improvement"],
            y = np.random.rand(len(df_subset)),
            mode = 'markers',
            customdata=df_plot["Configuration Number"],
            hovertemplate="%{customdata}",
            marker = dict(
                size = 10,
                color=df_plot["32-bit %"],
                line_width = 1,
                line_color = "black",
                opacity = 0.6,
                coloraxis="coloraxis1",
                symbol = "diamond",
            ),
            showlegend=False
        ),
        i + 1,
        1
    )
    if i == 0:
        fig.update_traces(
            marker_colorbar_title = "% 32-bit",
            marker_colorscale = "Plasma",
        )


    # df_plot = df_plot[df_plot["Configuration Number"] == df_entire[df_entire["Cost"] == df_entire[df_entire["Label"] == "Passed"]["Cost"].min()]["Configuration Number"].values[0]]
    # fig.add_trace(
    #     go.Scatter(
    #         x = df_plot["Improvement"],
    #         y = [0],
    #         mode = 'markers',
    #         customdata=[df_plot["Configuration Number"]],
    #         hovertemplate="%{customdata}",
    #         marker = dict(
    #             size = 15,
    #             color=df_plot["32-bit %"],
    #             line_width= 1,
    #             line_color="black",
    #             coloraxis="coloraxis1",
    #             symbol="star",
    #         ),
    #         name = "Optimal Variant",
    #         showlegend=[True if i == 0 else False for x in range(1)][0],
    #     ),
    #     row = i + 1,
    #     col = 1,
    # )

fig.update_layout(
    width = 600,
    height = 400,
    showlegend = False,
    margin = dict(
        r = 0,
        t = 60,
        b = 100,
        l = 0,
    ),
    title=dict(
        text = "MOM6",
        y = 0.98,
        x = 0.5,
        xanchor = 'center',
        yanchor = 'top',
        font_size=30,
    ),
    font_size = 24,
    font_family = "Times New Roman",
    coloraxis={
        "cmin" : 0,
        "cmax" : 100,
        "colorbar" : {
            'thickness' : 18,
            'title' : "% 32-bit<br>(Procedure)",
            'orientation' : 'h',
            'title_font_size' : 24,
            'y' : -0.6,
            'len' : 0.8
        },
    },
    legend = dict(
        bgcolor = "#E5ECF6",
        entrywidth = 130,
        orientation = "h",
        yanchor = "bottom",
        xanchor = "right",
        x = 1.0,
        y = 1.02,
        title_text = ""
    ),
)

fig.update_xaxes(type="log", ticksuffix="x")
# fig.update_xaxes(title="Speedup", row=len(baseline_costs), col=1)
fig.update_yaxes(visible=False)
fig.update_annotations(yshift=-5, font_size=18)
# fig.add_vline(x=1.0, line_width=2, line_dash="dash", line_color="grey")
fig.show()

In [51]:
df_plot = df_subset[df_subset["Procedure Name"] == "::mom_continuity_ppm::zonal_flux_adjust"]
# baseline_cost = baseline_costs[procedure_name] / df_plot[df_plot["Configuration Number"] == 0]["Execution Count"].values[0]
# baseline_cost = (baseline_cost + df_plot[df_plot["Configuration Number"] == 0]["Average CPU Time Per Call"].values[0]) / 2
baseline_cost = df_plot[df_plot["Configuration Number"] == 0]["Average CPU Time Per Call"].values[0]
df_plot = df_plot.assign(Improvement = np.round(baseline_cost/df_plot['Average CPU Time Per Call'], decimals=2))


In [59]:
df_plot.loc[df_plot["Improvement"] > 0.1, "Improvement"] = np.round(df_plot[df_plot["Improvement"] > 0.1]["Improvement"], decimals=1)
df_plot[df_plot["Improvement"] > 0.1]["Improvement"]


10      1.0
31      1.1
58      0.7
78      0.9
100     0.9
       ... 
7881    0.9
7903    0.6
7926    0.7
7974    0.8
7994    0.8
Name: Improvement, Length: 381, dtype: float64

10      1.0
31      1.1
58      0.7
78      0.9
100     0.9
       ... 
7881    0.9
7903    0.6
7926    0.7
7974    0.8
7994    0.8
Name: Improvement, Length: 381, dtype: float64

In [13]:
df_subset[(df_subset["Configuration Number"] == 25) & (df_subset["Procedure Name"] == "::mom_continuity_ppm::zonal_flux_adjust")]

,Configuration Number,Configuration Path,32-bit %,Label,Energy/Mass (L2 norm),Maximum CFL (L2 norm),Mean Sea Level (L2 norm),Total Mass (L2 norm),Mean Salin (L2 norm),Frac Mass Err (L2 norm),...,Mean Salin (inf norm),Frac Mass Err (inf norm),Salin Err (inf norm),Temp Err (inf norm),Cost,Procedure Name,CPU Time,Execution Count,config_hash,Average CPU Time Per Call
274,25,./prose_logs/0025/config_FAILED,67.741935,Exceeded Error Threshold,0.011517,0.275678,0.0,0.0,0.0,61.223669,...,0.0,51.27027,160.643836,0.169877,23.929,::mom_continuity_ppm::zonal_flux_adjust,1.588852,32000000.0,-633756409881252446,4.675324e-08


In [16]:
df_subset[(df_subset["Configuration Number"] == 575) & (df_subset["Procedure Name"] == "::mom_continuity_ppm::zonal_flux_adjust")]

,Configuration Number,Configuration Path,32-bit %,Label,Energy/Mass (L2 norm),Maximum CFL (L2 norm),Mean Sea Level (L2 norm),Total Mass (L2 norm),Mean Salin (L2 norm),Frac Mass Err (L2 norm),...,Mean Salin (inf norm),Frac Mass Err (inf norm),Salin Err (inf norm),Temp Err (inf norm),Cost,Procedure Name,CPU Time,Execution Count,config_hash,Average CPU Time Per Call
5386,575,./prose_logs/0575/config_PASSED,9.677419,Passed,0.01501,0.195036,0.0,0.0,0.0,109.112806,...,0.0,100.72973,176.39726,0.423729,7.009,::mom_continuity_ppm::zonal_flux_adjust,0.556938,7100000.0,4833932883432278975,6.878336e-08


In [15]:
df_subset[(df_subset["Configuration Number"] == 0) & (df_subset["Procedure Name"] == "::mom_continuity_ppm::zonal_flux_adjust")]

,Configuration Number,Configuration Path,32-bit %,Label,Energy/Mass (L2 norm),Maximum CFL (L2 norm),Mean Sea Level (L2 norm),Total Mass (L2 norm),Mean Salin (L2 norm),Frac Mass Err (L2 norm),...,Mean Salin (inf norm),Frac Mass Err (inf norm),Salin Err (inf norm),Temp Err (inf norm),Cost,Procedure Name,CPU Time,Execution Count,config_hash,Average CPU Time Per Call
10,0,./prose_logs/0000/config_PASSED,0.0,Passed,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.31,::mom_continuity_ppm::zonal_flux_adjust,0.470016,7500000.0,3695226326670877943,2.840622e-07
